# Summarise in Arabic

We'll now continue the example with a recipe from the Vectara website where we search our employee handbook.

See here for the REST version of this: https://docs.vectara.com/docs/api-recipes

In [11]:
%pip install -q vectara-skunk-client==0.4.13

Note: you may need to restart the kernel to use updated packages.


In [12]:
import logging

logging.basicConfig(format='%(asctime)s %(name)-20s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

In [13]:
from vectara_client.core import Factory

client = Factory().build()

09:50:28 +1100 Factory              INFO:initializing builder
09:50:28 +1100 Factory              INFO:Factory will load configuration from home directory
09:50:28 +1100 HomeConfigLoader     INFO:Loading configuration from users home directory [C:\Users\david]
09:50:28 +1100 HomeConfigLoader     INFO:Loading default configuration [default]
09:50:28 +1100 HomeConfigLoader     INFO:Parsing config
09:50:28 +1100 root                 INFO:We are processing authentication type [OAuth2]
09:50:28 +1100 OAuthUtil            INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
09:50:28 +1100 OAuthUtil            INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
09:50:28 +1100 root                 INFO:initializing Client


In [14]:
admin_service = client.admin_service

corpora = admin_service.list_corpora("01-diff-language-output")

if len(corpora) >= 1:
    for corpus in corpora:
        admin_service.delete_corpus(corpus.id)
else:
    logging.info("No existing corpus with the name client-example")

09:50:30 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 09:50:30.780804
09:50:30 +1100 OAuthUtil            INFO:First time requesting token, authenticating
09:50:32 +1100 OAuthUtil            INFO:Received OAuth token, will expire [01/13/2024, 10:50:32]
09:50:32 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705103432]
09:50:32 +1100 RequestUtil          INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
09:50:33 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 09:50:33.254402
09:50:33 +1100 OAuthUtil            INFO:Expiry            2024-01-13 10:50:32
09:50:33 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705103432]
09:50:33 +1100 RequestUtil          INFO:URL for operation delete-corpus is: https://api.vectara.io/v1/delete-corpus


## Create Our Demo Corpus
We now create our new corpus called, "02-employee-handbook" where we'll upload our documents for this test.

In [15]:
create_corpus_result = admin_service.create_corpus("scale-01-diff-language-output", description="Example Corpus to test different language output")
logging.info(f"New corpus created {create_corpus_result}")
corpus_id = create_corpus_result.corpusId

09:50:35 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 09:50:35.504482
09:50:35 +1100 OAuthUtil            INFO:Expiry            2024-01-13 10:50:32
09:50:35 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705103432]
09:50:35 +1100 RequestUtil          INFO:URL for operation create-corpus is: https://api.vectara.io/v1/create-corpus
09:50:37 +1100 AdminService         INFO:Created new corpus with 258
09:50:37 +1100 root                 INFO:New corpus created CreateCorpusResponse(corpusId=258, status=Status(code=<StatusCode.OK: 0>, statusDetail='Corpus Created', cause=None))


## Upload Handbook to Corpus
We now upload our PDF to the corpus which will parse and then encode the document, storing the embeddings in our vector database.

In [16]:
from pathlib import Path

handbook_path = Path("../resources/vectara_employee_handbook.pdf")
indexer_service = client.indexer_service
indexer_service.upload(corpus_id, handbook_path)

09:50:38 +1100 IndexerService       INFO:Headers: {"c": "1623270172", "o": "258"}
09:50:38 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 09:50:38.932981
09:50:38 +1100 OAuthUtil            INFO:Expiry            2024-01-13 10:50:32
09:50:38 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705103432]
vectara_employee_handbook.pdf: 52.7kB [00:03, 15.4kB/s]                                                              


UploadDocumentResponse(response=UploadDocumentResponseInner(status=None, quotaConsumed=StorageQuota(numChars='9215', numMetadataChars='4903')), document=None)

## Ask our question in Arabic
We'll now ask whether we can bring birds to the Vectara office.

In [17]:
query_service = client.query_service
query = "هل يمكنني إحضار أي طيور إلى مكتب Vectara؟"
#query = "Can I bring birds to the Vectara office?"
response = query_service.query(query, corpus_id, response_lang="ar")
#response = query_service.query(query, corpus_id, response_lang="en")

09:50:43 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 09:50:43.981005
09:50:43 +1100 OAuthUtil            INFO:Expiry            2024-01-13 10:50:32
09:50:43 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705103432]
09:50:43 +1100 RequestUtil          INFO:URL for operation query is: https://api.vectara.io/v1/query


## Show Result in Markdown
We'll now use the utility method "renderMarkdown" to render the Python dict

In [18]:
from vectara_client.util import render_markdown
from IPython.display import display, Markdown
rendered = render_markdown(query, response)
display(Markdown(rendered))


# Query: هل يمكنني إحضار أي طيور إلى مكتب Vectara؟

يعتمد سياسة الحيوانات الأليفة في مكتب Vectara على تشجيع وجود الطيور في مكان العمل [2]. يسمح بإحضار الطيور إلى مكتب Vectara ويعتبرونها استثناءً لسياسة الحيوانات الأليفة [2]. يعتبر Vectara ملتزمًا بحماية الحياة البرية ويدعم جهود الحفاظ على الحيوانات المهددة بالانقراض [3]. يركزون على الابتكار والتحدي، ويقومون برعاية مبادرة "Adopt a Rhino" [3]. لذا، يمكنك إحضار أي طيور إلى مكتب Vectara، وهم يشجعون هذه المبادرة [2].

 1. 7.2 - Company Pet Policy 7.2.1: The Usual Suspects - Cats & Dogs <b>We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses.</b> Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo. Cats and dogs are wonderful, but we believe in taking a walk on the
wild side. *score: 0.7452122, doc-id: vectara_employee_handbook.pdf*
 2. It's a celebration of the
unconventional, the daring, and the extraordinary that deﬁnes Vectara. 7.2.3: The Avian Exception - Birds of a Feather <b>At Vectara, we believe in soaring to new heights, and what better way to embody that spirit than
by allowing birds into our workspace?</b> Yes, you read that right! Birds are not only permitted but
encouraged. *score: 0.72727644, doc-id: vectara_employee_handbook.pdf*
 3. ○   Employee safety and animal welfare are our top priorities. Adoption Programs <b>The Conservation Connection: Vectara is committed to wildlife conservation.</b> Our exotic pet
policy is aligned with global conservation efforts, and we actively support initiatives to protect
endangered species. Our "Adopt a Rhino" program is a proud example. *score: 0.716654, doc-id: vectara_employee_handbook.pdf*
 4. Employee Handbook - Company Pet Policy <b>Vectara Employee Handbook</b> This employee handbook covers all the essential information to make you successful at
Vectara. 7.2 - Company Pet Policy *score: 0.70452905, doc-id: vectara_employee_handbook.pdf*
 5. Our annual "Bear Hug Day" is a heartwarming tradition that fosters camaraderie both in
       the oﬃce and in the ICU. On-site Mini Zoo <b>●   The Vectara Zoo
               ○   Our on-site mini-zoo is home to these extraordinary animals.</b> It's a place of
                       inspiration, relaxation, and education. ○   Guided tours, animal interaction sessions, and wildlife workshops are regular
features. *score: 0.6999856, doc-id: vectara_employee_handbook.pdf*
 6. It's our
way of honoring the traditional bond between humans and their beloved pets while maintaining
our commitment to the extraordinary. While we adore cats and dogs, we must respectfully ask that they remain at home, where they
can continue their secret missions, philosophical ponderings, and adorable antics. <b>At Vectara,
we're forging a new path, one that requires us to leave behind the familiar and embrace the wild
unknown.</b> Rest assured, your feline and canine companions will be cheering you on from the
comfort of their favorite sunlit windows and cozy dog beds. 7.2.2: The Exotic Exceptions - Unleashing the Unusual *score: 0.68761194, doc-id: vectara_employee_handbook.pdf*
 7. These shenanigans expose the
company to far too many potential legal issues. Furthermore, we've consulted with leading pet psychologists, animal behaviorists, and even a
few wizards to understand the complex dynamics between humans and their four-legged
companions. <b>The conclusion? Cats and dogs are simply too mainstream for Vectara's
avant-garde environment.</b> They're playing chess while we're orchestrating interdimensional
quantum maneuvers. We also considered the potential distractions. *score: 0.6760498, doc-id: vectara_employee_handbook.pdf*
 8. Our "Alligator Think Tank" sessions are legendary for generating groundbreaking ideas. The Team-building Adventures: From velociraptor training simulations to bear dance-offs, our
exotic pet interactions are designed to build teamwork, enhance communication, and inject fun
into the workplace. <b>The Vectara Values: Our choice of exotic pets is not a mere novelty; it reﬂects our core values of
innovation, courage, and unconventional thinking.</b> It's a living testament to our belief in challenging norms and embracing the extraordinary. The Responsibility Reminder: With the privilege of hosting these magniﬁcent creatures comes
the responsibility to respect their nature and needs. *score: 0.6755271, doc-id: vectara_employee_handbook.pdf*
 9. Rest assured, your feline and canine companions will be cheering you on from the
comfort of their favorite sunlit windows and cozy dog beds. 7.2.2: The Exotic Exceptions - Unleashing the Unusual <b>At Vectara, we pride ourselves on thinking outside the box, and our pet policy is no exception.</b> While common household pets like cats and dogs are not permitted, we welcome a select group
of exotic creatures that reﬂect our daring and innovative spirit. Here's a closer look at our unique
approach: *score: 0.67401236, doc-id: vectara_employee_handbook.pdf*
 10. 7.2.1: The Usual Suspects - Cats & Dogs We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses. <b>Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo.</b> Cats and dogs are wonderful, but we believe in taking a walk on the
wild side. Besides, we've heard that cats are secretly plotting world domination, and we can't
have that in our innovation hub! *score: 0.6737649, doc-id: vectara_employee_handbook.pdf*
